### 0. Import necessary libraries and methods

In [28]:
from project.preprocess_data import clean_data, parse_categories
from project.linear_regression import hashed_reg
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

### 1. Import data

In [29]:
dataset = pd.read_csv("Data/anpc-2016.csv")

/tmp/ipykernel_88010/1922025828.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Data/anpc-2016.csv")


### 2. Data preparation

In [30]:
clean_dataset = parse_categories(clean_data(dataset))

df initial length:  121187
-9.002235449  : <class 'numpy.float64'>
38.68091202  : <class 'numpy.float64'>
2016-01-09 14:02:00  : <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-01-09 17:30:00  : <class 'pandas._libs.tslibs.timestamps.Timestamp'>
df length:  118168


### 3. Data modelling

In [31]:
y_predicted = hashed_reg(occurrences=clean_dataset)

### 4. Evaluation

In [32]:
r2_score(clean_dataset['hh'], y_predicted)

0.0002740760194571701